In [1]:
import os
import openai
import pandas as pd
import numpy as np
import tiktoken
from dotenv import load_dotenv
from openai import OpenAI
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    f1_score,
    precision_score
)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

#### Necessary pip installation commands include:
- pip install os
- pip install openai
- pip install pandas
- pip install numpy
- pip install python-dotenv


#### <b>Note</b>: have a <b>.env</b> file already created for accessing API key

In [2]:
# retrieving dataset utilized for evaluation
df = pd.read_csv('../../data/csv/semi_toxic_classification_evaluation_set.csv')
df['Toxic'] = df['Toxic'].apply(lambda x: str(np.int64(x)))
df

,Text,Toxic
0,Date When did this algorithm get invented ?,0
1,answered them many times ! but they didnt answ...,0
2,IAC There is a discussion on article 's talk p...,0
3,Spandam 's laugh ? I 'm listening to it and al...,0
4,AND HAVING A FUCKIN ' BITCH ASS DAY '' ',1
...,...,...
995,"Oh , well Harry finally got his payback on me ...",1
996,this page is about random crap talk .. just ed...,1
997,I 've reverted the editsBOTH OF THEM VIOLATED ...,0
998,"I used the exact same bullshit you wrote , hyp...",1


In [3]:
# retrieving API Keyz from OpenAI platform
load_dotenv()
#tokenizer for GPT-3.5 Turbo
tokenizer = tiktoken.get_encoding("cl100k_base")
input_token_cost = ((0.50 / 1000000))
output_token_cost = ((1.50 / 1000000))
FINE_TUNED_TOXIC_DETECTION_API_KEY = os.getenv("FINE_TUNED_TOXIC_DETECTION_API_KEY")
REGULAR_TOXIC_DETECTION_API_KEY = os.getenv("REGULAR_TOXIC_DETECTION_API_KEY")

In [4]:
# instantiating client with API key
general = OpenAI(
    api_key=REGULAR_TOXIC_DETECTION_API_KEY
)

In [5]:
# creating container for holding model completions for comparison
original = pd.DataFrame(columns=['Text', 'Toxic'])
tokens = pd.DataFrame(columns=['Prompt', 'Input', 'Output', 'Total'])

In [6]:
# processing original gpt-3.5-turbo-0125 model across entire dataset
for index, row in df.iterrows():
    completion = general.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {
                "role": "system", 
                "content": "Your job is to detect toxicity from a non-biased viewpoint and label prompted content as either 1 for toxic or 0 for non-toxic"
            },
            {
                "role": "user",
                "content": row['Text']
            }
        ]
    )
    original.loc[index] = [row['Text'], completion.choices[0].message.content]
    tokens_object = {
        'prompt': len(tokenizer.encode("Your job is to detect toxicity from a non-biased viewpoint and label prompted content as either 1 for toxic or 0 for non-toxic")),
        'input': len(tokenizer.encode(f"Your job is to detect toxicity from a non-biased viewpoint and label prompted content as either 1 for toxic or 0 for non-toxic. {row['Text']}")),
        'output': len(tokenizer.encode(completion.choices[0].message.content))
    }
    total_tokens = tokens_object['input'] + tokens_object['output']
    tokens.loc[index] = [tokens_object['prompt'], tokens_object['input'], tokens_object['output'], total_tokens]


In [7]:
tokens

,Prompt,Input,Output,Total
0,30,39,1,40
1,30,51,1,52
2,30,44,1,45
3,30,62,1,63
4,30,44,1,45
...,...,...,...,...
995,30,197,1,198
996,30,43,3,46
997,30,49,1,50
998,30,43,1,44


In [8]:
# gpt-3.5 needs prompt to be really specific; more instruction needed
# gpt-4 will give you a response of a number as requested, gpt-3.5 will give
# a longer sequence of characters which wasn't asked for
original

,Text,Toxic
0,Date When did this algorithm get invented ?,0
1,answered them many times ! but they didnt answ...,1
2,IAC There is a discussion on article 's talk p...,0
3,Spandam 's laugh ? I 'm listening to it and al...,0
4,AND HAVING A FUCKIN ' BITCH ASS DAY '' ',1
...,...,...
995,"Oh , well Harry finally got his payback on me ...",1
996,this page is about random crap talk .. just ed...,1 - toxic
997,I 've reverted the editsBOTH OF THEM VIOLATED ...,1
998,"I used the exact same bullshit you wrote , hyp...",1


In [9]:
# comparing results of model to dataset
compare = original['Toxic'] == df['Toxic']
gptthreefive_original_accuracy = compare.values.sum() / compare.size

In [10]:
compare

0       True
1      False
2       True
3       True
4       True
       ...  
995     True
996    False
997    False
998     True
999     True
Name: Toxic, Length: 1000, dtype: bool

In [11]:
# gpt-3.5-turbo-0125 model accuracy with role prompting technique 74.00%
print(f"Accuracy: {gptthreefive_original_accuracy * 100:.2f}%")

Accuracy: 74.00%


In [12]:
# total token input cost: $0.049638499999999995
(tokens['Input'].sum() * input_token_cost)

0.049638499999999995

In [13]:
# total token output cost: $0.0029460000000000003
(tokens['Output'].sum() * output_token_cost)

0.0029460000000000003

In [14]:
# complete total token cost: $0.05258449999999999
(tokens['Input'].sum() * input_token_cost) + (tokens['Output'].sum() * output_token_cost)

0.05258449999999999

In [15]:
# 101241
tokens['Total'].sum()

101241